# FastF1 Lap Dataset Builder

Use [FastF1](https://theoehrly.github.io/Fast-F1/) to download official timing data (laps, weather, race control, telemetry) and build a driver/lap level dataset enriched with tyre compounds, DRS, weather, and safety car indicators.

## Requirements

* `pip install fastf1`
* Configure a cache directory. FastF1 caches the raw livetiming files, so reuse the cache to avoid repeated downloads.
* An internet connection (FastF1 talks to the official F1 API).

In [1]:
%pip install fastf1

Note: you may need to restart the kernel to use updated packages.


In [1]:
import fastf1 as ff1
import pandas as pd
from pathlib import Path

NOTEBOOK_DIR = Path.cwd()
if not (NOTEBOOK_DIR / 'fastf1_lap_dataset.ipynb').exists():
    NOTEBOOK_DIR = NOTEBOOK_DIR / 'models'
NOTEBOOK_DIR.mkdir(parents=True, exist_ok=True)

cache_dir = NOTEBOOK_DIR / '.fastf1_cache'
cache_dir.mkdir(parents=True, exist_ok=True)
ff1.Cache.enable_cache(str(cache_dir.resolve()))
ff1.Cache.offline_mode = True
ff1.set_log_level('DEBUG')

In [2]:
ff1.Cache.get_cache_info()

('/Users/ekazuki/Documents/f1stuff/models/.fastf1_cache', 1187871768)

In [3]:
MIN_YEAR = 2018
MAX_YEAR = 2025  # Fetch every season from 2018 through 2024
SESSION_TYPES = ['Race', 'Sprint']

OUTPUT_PATH = NOTEBOOK_DIR / 'fastf1_lap_dataset2.csv'
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

In [4]:
def collect_sessions(year):
    print(f"Collecting sessions for {year}...")
    schedule = ff1.get_event_schedule(year)
    for _, event in schedule.iterrows():
        for session_name in SESSION_TYPES:
            try:
                session = ff1.get_session(year, event['EventName'], session_name)
            except Exception as exc:
                print(f"Skipping {year} {event['EventName']} {session_name}: {exc}")
                continue
            yield session

In [5]:
def build_lap_frame(session):
    session.load(laps=True, telemetry=False, weather=True, messages=True)
    laps = session.laps.copy()
    laps['LapTimeSeconds'] = laps['LapTime'].dt.total_seconds()
    laps['GapToLeaderSeconds'] = laps['Time'] - laps.groupby('LapNumber')['Time'].transform('min')
    laps['GapToLeaderSeconds'] = laps['GapToLeaderSeconds'].dt.total_seconds()
    laps = laps.sort_values(['LapNumber', 'Time'])
    laps['GapToAheadSeconds'] = laps.groupby('LapNumber')['Time'].diff().dt.total_seconds().fillna(0)
    laps['Compound'] = laps.groupby(['Driver', 'Stint'])['Compound'].transform(lambda s: s.ffill().bfill())
    stint_info = laps[['Driver', 'Stint', 'LapNumber', 'Compound', 'TyreLife']]
    weather = session.weather_data.copy() if session.weather_data is not None else pd.DataFrame()
    if not weather.empty:
        weather['Timestamp'] = weather['Time'].dt.total_seconds()
    rc_messages = session.race_control_messages.copy() if session.race_control_messages is not None else pd.DataFrame()
    track_status = session.track_status.copy() if session.track_status is not None else pd.DataFrame()
    return laps, stint_info, weather, rc_messages, track_status


In [6]:
def safety_car_flag(lap_start, lap_end, track_status):
    if lap_start is None or lap_end is None:
        return 0, 0
    if track_status is None or track_status.empty:
        return 0, 0
    sc_codes = {'SC', 'SC Withdrawn', 'SafetyCar'}
    vsc_codes = {'VSC', 'VSC Ending', 'Virtual SafetyCar'}
    sc = 0
    vsc = 0
    for _, row in track_status.iterrows():
        status_time = row['Time'].total_seconds()
        message = str(row.get('Message', '')).upper()
        if lap_start <= status_time <= lap_end:
            if any(code in message for code in ['SAFETY', 'SC', 'CODE4']):
                sc = 1
            if 'VIRTUAL' in message or 'VSC' in message:
                vsc = 1
    return sc, vsc


In [7]:
def drs_flag(lap_start, lap_end, rc_messages):
    if lap_start is None or lap_end is None:
        return 0
    if rc_messages is None or rc_messages.empty:
        return 0
    enabled_times = []
    disabled_times = []
    for _, row in rc_messages.iterrows():
        if str(row.get('Category', '')).upper() != 'DRS':
            continue
        rel_time = row.get('Time')
        if rel_time is None or pd.isna(rel_time):
            continue
        if hasattr(rel_time, 'total_seconds'):
            ts = rel_time.total_seconds()
        elif hasattr(rel_time, 'timestamp'):
            ts = rel_time.timestamp()
        else:
            try:
                ts = float(rel_time)
            except (TypeError, ValueError):
                continue
        message = str(row.get('Message', '')).upper()
        if 'ENABLE' in message:
            enabled_times.append(ts)
        elif 'DISABLE' in message:
            disabled_times.append(ts)
    if not enabled_times:
        return 0
    eligible_enables = [t for t in enabled_times if t <= lap_end]
    if not eligible_enables:
        return 0
    enable_limit = max(eligible_enables)
    disable_after = min((t for t in disabled_times if t >= enable_limit), default=None)
    if disable_after is not None and disable_after < lap_start:
        return 0
    return 1

In [8]:
def slugify(value: str) -> str:
    if not isinstance(value, str):
        return ''
    cleaned = ''.join(ch.lower() if ch.isalnum() else '_' for ch in value)
    return '_'.join(part for part in cleaned.split('_') if part)


def rainfall_indicator(value):
    if value is None:
        return pd.NA
    try:
        if pd.isna(value):
            return pd.NA
    except TypeError:
        pass
    if isinstance(value, str):
        lowered = value.strip().lower()
        if not lowered:
            return pd.NA
        if lowered in {'no', 'dry', 'false'}:
            return False
        if lowered in {'yes', 'wet', 'rain', 'true'}:
            return True
        try:
            numeric = float(lowered)
        except ValueError:
            return pd.NA
        return numeric > 0
    try:
        numeric = float(value)
    except (TypeError, ValueError):
        return pd.NA
    return numeric > 0


def timedelta_to_seconds(value):
    if value is None:
        return None
    try:
        if pd.isna(value):
            return None
    except TypeError:
        pass
    if hasattr(value, 'total_seconds'):
        return value.total_seconds()
    try:
        return float(value)
    except (TypeError, ValueError):
        return None


rows = []
for year in range(MIN_YEAR, MAX_YEAR + 1):
    for session in collect_sessions(year):
        print(f"Loading {session.event['EventName']} {session.name}")
        try:
            laps, _, weather, rc_messages, track_status = build_lap_frame(session)
        except Exception as exc:
            print(f"Failed to load {session.event['EventName']} {session.name}: {exc}")
            continue

        driver_meta = getattr(session, 'results', None)
        has_driver_meta = isinstance(driver_meta, pd.DataFrame) and {'DriverNumber', 'GridPosition'}.issubset(driver_meta.columns)
        if has_driver_meta:
            driver_meta = driver_meta[['DriverNumber', 'GridPosition']].copy()
            driver_meta['DriverNumber'] = driver_meta['DriverNumber'].astype(str)
            laps['DriverNumber'] = laps['DriverNumber'].astype(str)
            driver_meta = driver_meta.rename(columns={'GridPosition': 'GridPositionResult'})
            laps = laps.merge(driver_meta, on='DriverNumber', how='left')
            laps['grid_position'] = laps['GridPositionResult']
            laps = laps.drop(columns=['GridPositionResult'])
        else:
            laps['grid_position'] = pd.NA

        if 'StintLap' not in laps.columns:
            laps['StintLap'] = laps.groupby(['Driver', 'Stint']).cumcount() + 1

        total_laps = session.total_laps or int(laps['LapNumber'].max())
        session_identifier = getattr(session, 'session_key', None)
        if not session_identifier:
            session_identifier = slugify(f"{session.date.year}_{session.event['EventName']}_{session.name}")
        circuit_source = session.event.get('Location') or session.event.get('EventName') or ''
        circuit_slug = slugify(circuit_source)
        team_slug_cache = {}

        for _, lap in laps.iterrows():
            is_pit_in = lap.get('PitInTime', False)
            is_pit_out = lap.get('PitOutTime', False)
            is_pit = pd.notna(is_pit_in) or pd.notna(is_pit_out)

            team_name = lap.get('Team')
            if team_name not in team_slug_cache:
                team_slug_cache[team_name] = slugify(team_name) if isinstance(team_name, str) else ''
            lap_start = timedelta_to_seconds(lap.get('LapStartTime'))
            lap_end = timedelta_to_seconds(lap.get('Time'))
            sc_flag, vsc_flag = safety_car_flag(lap_start, lap_end, track_status)
            drs_active = drs_flag(lap_start, lap_end, rc_messages)
            weather_slice = {}
            if weather is not None and not weather.empty and lap_end is not None:
                idx = (weather['Timestamp'] - lap_end).abs().argsort()[:1]
                weather_slice = weather.iloc[idx].to_dict('records')[0]
            rainfall_value = weather_slice.get('Rainfall', pd.NA)
            rows.append({
                'session_key': session_identifier,
                'race_name': session.event['EventName'],
                'year': session.date.year,
                'session_name': session.name,
                'circuit_id': circuit_slug,
                'driver_id': lap['Driver'],
                'team_id': team_slug_cache.get(team_name, ''),
                'team_name': team_name,
                'lap_number': lap['LapNumber'],
                'total_race_laps': total_laps,
                'grid_position': lap.get('grid_position'),
                'current_position': lap.get('Position'),
                'lap_time_s': lap['LapTimeSeconds'],
                'gap_to_leader_s': lap['GapToLeaderSeconds'],
                'gap_to_ahead_s': lap['GapToAheadSeconds'],
                'laps_on_current_tyre': lap['TyreLife'] if pd.notna(lap['TyreLife']) else lap['StintLap'],
                'tyre_compound': lap.get('Compound'),
                'drs_enabled': drs_active,
                'safety_car_this_lap': sc_flag,
                'virtual_sc_this_lap': vsc_flag,
                'air_temperature': weather_slice.get('AirTemp'),
                'track_temperature': weather_slice.get('TrackTemp'),
                'humidity': weather_slice.get('Humidity'),
                'pressure': weather_slice.get('Pressure'),
                'rainfall': weather_slice.get('Rainfall'),
                'wind_speed': weather_slice.get('WindSpeed'),
                'wind_direction': weather_slice.get('WindDirection'),
                'has_rain': rainfall_indicator(rainfall_value),
                'is_pit': is_pit,
            })

Loading Australian Grand Prix Race


core           INFO 	Loading data for Australian Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '28'
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 5 completed the race distance 00:00.123000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['5', '44', '7', '3', '14', '33', '27', '77', '2', '55', '11', '31', '16', '18', '28', '8', '20', 

Skipping 2018 Australian Grand Prix Sprint: Session type 'Sprint' does not exist for this event
Loading Bahrain Grand Prix Race


req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['5', '77', '44', '10', '20', '27', '14', '2', '9', '31', '55', '16', '8', '18', '35', '11', '28', '7', '33', '3']
core           INFO 	Loading data for Chinese Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '3'
core        WARNING 	Fixed incorrect tyre stint information for driver '33'
core        WARNING 	F

Skipping 2018 Bahrain Grand Prix Sprint: Session type 'Sprint' does not exist for this event
Loading Chinese Grand Prix Race


...

Loading Qatar Grand Prix Sprint


core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


Loading Abu Dhabi Grand Prix Race


req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '81', '4', '16', '63', '14', '31', '44', '27', '18', '5', '87', '55', '22', '12', '23', '6', '30', '10', '43']


Skipping 2025 Abu Dhabi Grand Prix Sprint: Session type 'Sprint' does not exist for this event


In [10]:
fastf1_df = pd.DataFrame(rows)
if fastf1_df.empty:
    raise ValueError('No laps collected. Adjust MIN_YEAR/MAX_YEAR or SESSION_TYPES.')

integer_columns = ['lap_number', 'total_race_laps', 'grid_position', 'current_position', 'laps_on_current_tyre']
for col in integer_columns:
    fastf1_df[col] = pd.to_numeric(fastf1_df[col], errors='coerce').astype('Int64')

fastf1_df['tyre_compound'] = fastf1_df.groupby(['session_key', 'driver_id'])['tyre_compound'].transform(lambda s: s.ffill().bfill())

boolean_columns = ['drs_enabled', 'safety_car_this_lap', 'virtual_sc_this_lap', 'has_rain', 'is_pit']
for col in boolean_columns:
    fastf1_df[col] = fastf1_df[col].astype('boolean')

fastf1_df = fastf1_df.sort_values(['year', 'session_key', 'driver_id', 'lap_number']).reset_index(drop=True)

dedup_subset = ['session_key', 'driver_id', 'lap_number']
before_dedup = len(fastf1_df)
fastf1_df = fastf1_df.drop_duplicates(subset=dedup_subset, keep='first')
after_dedup = len(fastf1_df)
if after_dedup != before_dedup:
    print(f'Removed {before_dedup - after_dedup} duplicate driver/lap rows')
fastf1_df = fastf1_df.reset_index(drop=True)

base_columns = [
    'driver_id',
    'team_id',
    'circuit_id',
    'total_race_laps',
    'year',
    'session_name',
    'grid_position',
    'current_position',
    'gap_to_leader_s',
    'gap_to_ahead_s',
    'lap_time_s',
    'laps_on_current_tyre',
    'tyre_compound',
    'safety_car_this_lap',
    'lap_number',
    'drs_enabled',
    'track_temperature',
    'air_temperature',
    'has_rain',
    'is_pit'
]
extra_columns = [
    'session_key',
    'race_name',
    'team_name',
    'virtual_sc_this_lap',
    'humidity',
    'pressure',
    'rainfall',
    'wind_speed',
    'wind_direction',
]
ordered_columns = base_columns + [col for col in extra_columns if col in fastf1_df.columns]
fastf1_df = fastf1_df[ordered_columns]

fastf1_df.to_csv(OUTPUT_PATH, index=False)
fastf1_df.head()


/var/folders/3t/lccpd1j15071g5nr0s6sjcr80000gn/T/ipykernel_28815/1986097186.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fastf1_df['tyre_compound'] = fastf1_df.groupby(['session_key', 'driver_id'])['tyre_compound'].transform(lambda s: s.ffill().bfill())


Removed 8350 duplicate driver/lap rows


,driver_id,team_id,circuit_id,total_race_laps,year,session_name,grid_position,current_position,gap_to_leader_s,gap_to_ahead_s,...,is_pit,session_key,race_name,team_name,virtual_sc_this_lap,humidity,pressure,rainfall,wind_speed,wind_direction
0,ALO,mclaren,yas_marina,55,2018,Race,15,14,26.062,2.134,...,False,2018_abu_dhabi_grand_prix_race,Abu Dhabi Grand Prix,McLaren,False,38.7,1012.8,False,0.7,272
1,ALO,mclaren,yas_marina,55,2018,Race,15,14,16.996,1.680,...,False,2018_abu_dhabi_grand_prix_race,Abu Dhabi Grand Prix,McLaren,False,38.2,1012.7,False,1.5,225
2,ALO,mclaren,yas_marina,55,2018,Race,15,14,16.340,0.813,...,False,2018_abu_dhabi_grand_prix_race,Abu Dhabi Grand Prix,McLaren,False,37.5,1012.7,False,1.1,268
3,ALO,mclaren,yas_marina,55,2018,Race,15,14,8.010,0.432,...,False,2018_abu_dhabi_grand_prix_race,Abu Dhabi Grand Prix,McLaren,False,37.7,1012.8,False,0.8,267
4,ALO,mclaren,yas_marina,55,2018,Race,15,14,12.847,0.613,...,False,2018_abu_dhabi_grand_prix_race,Abu Dhabi Grand Prix,McLaren,False,37.5,1012.8,False,1.3,0


In [44]:
race = ff1.get_session(2024, 'Monza', 'Race')
race.load(laps=True, telemetry=False, weather=True, messages=True)
race.laps[10:20]

core           INFO 	Loading data for Italian Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '81', '4', '55', '44', '1', '63', '11', '23', '20', '14', '43', '3', '31', '10', '77', '27', '24', '18', '22']


,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
10,0 days 01:11:26.225000,LEC,16,0 days 00:01:24.846000,11.0,1.0,NaT,NaT,0 days 00:00:27.633000,0 days 00:00:28.951000,...,True,Ferrari,0 days 01:10:01.379000,NaT,1,2.0,False,,False,True
11,0 days 01:12:51.141000,LEC,16,0 days 00:01:24.916000,12.0,1.0,NaT,NaT,0 days 00:00:27.680000,0 days 00:00:28.971000,...,True,Ferrari,0 days 01:11:26.225000,NaT,1,2.0,False,,False,True
12,0 days 01:14:16.670000,LEC,16,0 days 00:01:25.529000,13.0,1.0,NaT,NaT,0 days 00:00:27.812000,0 days 00:00:29.177000,...,True,Ferrari,0 days 01:12:51.141000,NaT,1,2.0,False,,False,True
13,0 days 01:15:42.276000,LEC,16,0 days 00:01:25.606000,14.0,1.0,NaT,NaT,0 days 00:00:27.839000,0 days 00:00:29.217000,...,True,Ferrari,0 days 01:14:16.670000,NaT,1,2.0,False,,False,True
14,0 days 01:17:12.468000,LEC,16,0 days 00:01:30.192000,15.0,1.0,NaT,0 days 01:17:08.772000,0 days 00:00:28.034000,0 days 00:00:29.361000,...,True,Ferrari,0 days 01:15:42.276000,NaT,1,3.0,False,,False,False
15,0 days 01:18:57.356000,LEC,16,0 days 00:01:44.888000,16.0,2.0,0 days 01:17:33.191000,NaT,0 days 00:00:47.985000,0 days 00:00:28.750000,...,True,Ferrari,0 days 01:17:12.468000,NaT,1,7.0,False,,False,False
16,0 days 01:20:21.626000,LEC,16,0 days 00:01:24.270000,17.0,2.0,NaT,NaT,0 days 00:00:27.458000,0 days 00:00:28.730000,...,True,Ferrari,0 days 01:18:57.356000,NaT,1,6.0,False,,False,True
17,0 days 01:21:45.501000,LEC,16,0 days 00:01:23.875000,18.0,2.0,NaT,NaT,0 days 00:00:27.408000,0 days 00:00:28.571000,...,True,Ferrari,0 days 01:20:21.626000,NaT,1,6.0,False,,False,True
18,0 days 01:23:09.704000,LEC,16,0 days 00:01:24.203000,19.0,2.0,NaT,NaT,0 days 00:00:27.354000,0 days 00:00:28.722000,...,True,Ferrari,0 days 01:21:45.501000,NaT,1,6.0,False,,False,True
19,0 days 01:24:33.530000,LEC,16,0 days 00:01:23.826000,20.0,2.0,NaT,NaT,0 days 00:00:27.483000,0 days 00:00:28.623000,...,True,Ferrari,0 days 01:23:09.704000,NaT,1,5.0,False,,False,True


In [43]:
import numpy as np

print(math.isnan(race.laps.iloc[0].PitInTime))
race.laps.iloc[0]

NameError: name 'math' is not defined

In [ ]:
fastf1_df["is_pit"]

count     198633
unique         2
top        False
freq      186420
Name: is_pit, dtype: object

0         False
1         False
2         False
3         False
4         False
          ...  
198628    False
198629    False
198630    False
198631    False
198632    False
Name: is_pit, Length: 198633, dtype: boolean